# JHU CSSE GIS Dashboard
* https://www.arcgis.com/apps/opsdashboard/index.html#/bda7594740fd40299423467b48e9ecf6
* https://coronavirus.jhu.edu/map.html
* https://www.arcgis.com/home/item.html?id=bbb2e4f589ba40d692fab712ae37b9ac#data

In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

In [2]:
data = requests.get(
    'https://services1.arcgis.com/0MSEUqKaxRlEPj5g/arcgis/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/2/query?f=json&where=1%3D1&returnGeometry=false&spatialRel=esriSpatialRelIntersects&outFields=*&orderByFields=OBJECTID%20ASC&resultOffset=0&resultRecordCount=200&cacheHint=true&quantizationParameters=%7B%22mode%22%3A%22edit%22%7D').json()

In [3]:
country_list = []
for country in data['features']:
    country_dic = {}
    country_dic['OBJECTID'] = country['attributes']['OBJECTID']
    country_dic['Country_Region'] = country['attributes']['Country_Region']
    country_dic['Last_Update'] = country['attributes']['Last_Update']
    country_dic['Lat'] = country['attributes']['Lat']
    country_dic['Long_'] = country['attributes']['Long_']
    country_dic['Confirmed'] = country['attributes']['Confirmed']
    country_dic['Deaths'] = country['attributes']['Deaths']
    country_dic['Recovered'] = country['attributes']['Recovered']
    country_list.append(country_dic)

In [4]:
df = pd.DataFrame(country_list)

In [5]:
df.head()

,OBJECTID,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered
0,1,Australia,1585723619000,-25.0000,133.0000,4709,19,358
1,2,Austria,1585723235000,47.5162,14.5501,10192,128,1095
2,3,Canada,1585723605000,60.0010,-95.0010,8591,102,1592
3,4,China,1585719413000,30.5928,114.3055,82301,3310,76207
4,5,Denmark,1585723235000,56.0000,10.0000,3039,90,77


In [6]:
df.Last_Update = pd.to_datetime(
    df.Last_Update/1000,unit='s').dt.tz_localize('utc').dt.tz_convert('Asia/Tokyo').dt.strftime('%Y-%m-%d %H:%M')

In [7]:
df.head()

,OBJECTID,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered
0,1,Australia,2020-04-01 15:46,-25.0000,133.0000,4709,19,358
1,2,Austria,2020-04-01 15:40,47.5162,14.5501,10192,128,1095
2,3,Canada,2020-04-01 15:46,60.0010,-95.0010,8591,102,1592
3,4,China,2020-04-01 14:36,30.5928,114.3055,82301,3310,76207
4,5,Denmark,2020-04-01 15:40,56.0000,10.0000,3039,90,77


In [8]:
df['Active'] = df.Confirmed - df.Deaths - df.Recovered

In [9]:
df.to_csv('data/latest_global_'+datetime.now().strftime('%Y%m%d_%I%M%p')+'.csv',index=False,encoding='utf-8-sig')

In [10]:
countries = pd.read_csv('countries.csv')

In [11]:
countries.head()

,Country/Region,Japanese,Code
0,Afghanistan,アフガニスタン,AF
1,Albania,アルバニア,AL
2,Algeria,アルジェリア,DZ
3,Andorra,アンドラ,AD
4,Angola,アンゴラ,AO


In [12]:
df.Country_Region = df.Country_Region.map(countries.set_index('Country/Region').Japanese.to_dict())

In [13]:
df.head()

,OBJECTID,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active
0,1,オーストラリア,2020-04-01 15:46,-25.0000,133.0000,4709,19,358,4332
1,2,オーストリア,2020-04-01 15:40,47.5162,14.5501,10192,128,1095,8969
2,3,カナダ,2020-04-01 15:46,60.0010,-95.0010,8591,102,1592,6897
3,4,中国,2020-04-01 14:36,30.5928,114.3055,82301,3310,76207,2784
4,5,デンマーク,2020-04-01 15:40,56.0000,10.0000,3039,90,77,2872


In [14]:
df.columns = ['ID','国名','更新時間（日本時間）','緯度','経度','累積感染者数','死者数','回復者数','現行感染者数']

In [15]:
df.head()

,ID,国名,更新時間（日本時間）,緯度,経度,累積感染者数,死者数,回復者数,現行感染者数
0,1,オーストラリア,2020-04-01 15:46,-25.0000,133.0000,4709,19,358,4332
1,2,オーストリア,2020-04-01 15:40,47.5162,14.5501,10192,128,1095,8969
2,3,カナダ,2020-04-01 15:46,60.0010,-95.0010,8591,102,1592,6897
3,4,中国,2020-04-01 14:36,30.5928,114.3055,82301,3310,76207,2784
4,5,デンマーク,2020-04-01 15:40,56.0000,10.0000,3039,90,77,2872


In [16]:
summary = df[['累積感染者数','死者数','回復者数','現行感染者数']].sum().reset_index()

In [17]:
summary.columns = ['項目','合計']

In [18]:
summary = summary.set_index('項目').T

In [19]:
summary.index = [datetime.today().strftime('%Y-%m-%d')]

In [20]:
summary

項目,累積感染者数,死者数,回復者数,現行感染者数
2020-04-01,860793,42354,178378,640061


In [21]:
total = pd.read_excel('covid19_'+datetime.today().strftime('%Y%m%d')+'.xlsx',sheet_name='合計',index_col=0
                     ).append(summary,sort=True).sort_index(ascending=False)

In [22]:
total.head()

,回復者数,新規感染者数,死者数,現行感染者数,累積感染者数
2020-04-01,178378,NaN,42354,640061,860793
2020-03-31,178034,75122.0,42107,637346,857487
2020-03-30,164566,62248.0,37582,580217,782365
2020-03-29,149082,59411.0,33925,537110,720117
2020-03-28,139415,67415.0,30652,490639,660706


In [23]:
total.iloc[0,1] = total.iloc[0,4] - total.iloc[1,4]

In [24]:
total['新規感染者数'] = total['新規感染者数'].astype(int)

In [25]:
total.head()

,回復者数,新規感染者数,死者数,現行感染者数,累積感染者数
2020-04-01,178378,3306,42354,640061,860793
2020-03-31,178034,75122,42107,637346,857487
2020-03-30,164566,62248,37582,580217,782365
2020-03-29,149082,59411,33925,537110,720117
2020-03-28,139415,67415,30652,490639,660706


In [26]:
with pd.ExcelWriter('latest_global_'+datetime.now().strftime('%Y%m%d_%I%M%p')+'.xlsx') as writer:  
    df.set_index('国名').sort_values('累積感染者数',ascending=False).drop(['ID'],axis=1).to_excel(writer,sheet_name='最新')
    total.to_excel(writer,sheet_name='合計')